In [8]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import sample

In [9]:
train_data = pd.read_csv('train.csv', sep=',')  # 训练数据
train_data_matrix = np.zeros((2967, 4125))  # 训练数据矩阵
for line in train_data.itertuples():
    train_data_matrix[line[1], line[2]] = line[3]

In [10]:
def get_dict_u():
    dict1 = {}#不同user各自对应的item
    dict2 = {}#不同item各自对应的user
    for i in range(2967):
        dict1[i] = []

        for j in range(4125):
            if train_data_matrix[i][j] != 0:
                dict1[i].append(j)
                if j not in dict2.keys():
                    dict2[j] = []
                dict2[j].append(i)
    return [dict1, dict2]

In [1]:
# K=10,alpha=0.01,lbd = 0.3,miu = 0.1,bu = 0.1,bm = 0.1; rsme = 0.93
# K = 30,alpha = 0.01,lbd = 0.3,miu = 0.1,bu = 0.1,bm = 0.1;rsme = 0.98
# K = 30，alpha = 0.01，lbd = 0.3，miu = 0.2，bu = 0.1，bm = 0.1；rsme = 0.91
# K = 40，alpha = 0.01，lbd = 0.3，miu = 0.3，bu = 0.1，bm = 0.1；rsme = 0.92
# K = 40,alpha = 0.01,lbd = 0.3,miu = 0.25,bu = 0.1bm = 0.1,;rsme = 0.92
# K = 20,alpha = 0.01,lbd = 0.3,miu = 0.2,bu = 0.1,bm = 0.1;rsme = 0.89
# K = 15,alpha = 0.01,lbd = 0.3,miu = 0.2,bu = 0.1,bm = 0.1;rsme = 0.898
# K = 10,alpha = 0.01,lbd = 0.3,miu = 0.2,bu = 0.1,bm = 0.1;for(35);rsme = 0.89
# K = 10,alpha = 0.01,lbd = 0.3,miu = 0.15,bu = 0.1,bm = 0.1;for(35);rsme = 0.90
# K = 30,alpha = 0.01,lbd = 0.3,miu = 0.15,bu = 0.1,bm = 0.1;for(20)；rsme = 0.89
# K = 10,alpha = 0.01,lbd = 0.3,miu = 0.15,bu = 0.1,bm = 0.1;for(35)

In [50]:
K = 30
alpha = 0.01
lbd = 0.3
miu = 0.2
bu = 0.1
bm = 0.1
F = np.random.rand(2967,K)
G = np.random.rand(4125,K)

lis = get_dict_u()

In [56]:
for z in tqdm(range(20)):
    for i in lis[0].keys():
        for j in lis[0][i]:
            F[i] = (1-alpha*lbd)*F[i] + alpha*G[j]*(train_data_matrix[i][j] - (miu + bu + bm + np.dot(F[i],(G[j].T))))
            G[j] = (1-alpha*lbd)*G[j] + alpha*F[i]*(train_data_matrix[i][j] - (miu + bu + bm + np.dot(F[i],(G[j].T))))
            bu = (1-alpha*lbd)*bu + alpha*(train_data_matrix[i][j] - (miu + bu + bm +np.dot(F[i],(G[j].T))))
            bm = (1-alpha*lbd)*bm + alpha*(train_data_matrix[i][j] - (miu + bu + bm + np.dot(F[i],(G[j].T))))
            # print(F[i])
            # print(G[j])
            # F[i] = fi
            # G[j] = gj
            # bu = bu_t
            # bm = bm_t
    print(F)
    print(G)

    print(np.dot(F,G.T) + miu + bu + bm)
    res = np.dot(F,G.T) + miu + bu + bm
    df = pd.read_csv("test_index.csv")
    test_data_matrix = df.values

    predict_matrix = np.zeros((39679, 2))

    f = open('predict2.csv','w')
    f.write('dataID,rating\n')
    for i in range(test_data_matrix.shape[0]):
        predict_matrix[i][0] = i
        predict_matrix[i][1] = res[test_data_matrix[i][0]][test_data_matrix[i][1]]
        f.write('%d,%.8f\n'%(predict_matrix[i][0],predict_matrix[i][1]))

  0%|          | 0/20 [00:00<?, ?it/s]

[[0.11696308 0.17312134 0.18805298 ... 0.25282806 0.22942513 0.1405728 ]
 [0.10571077 0.12687509 0.14727138 ... 0.18408553 0.15411249 0.14627632]
 [0.13513574 0.14705666 0.1623079  ... 0.12763915 0.14467535 0.11867155]
 ...
 [0.27460679 0.25263509 0.22905562 ... 0.21163861 0.23833268 0.23750583]
 [0.33975416 0.22264311 0.2447329  ... 0.36267033 0.2944282  0.23310791]
 [0.14387286 0.11794651 0.0872906  ... 0.11477544 0.14827497 0.21504723]]
[[ 0.17676218  0.19737613  0.23634819 ...  0.26768864  0.23895134
   0.24871037]
 [ 0.13973428  0.16296528  0.18420361 ...  0.16803012  0.18804381
   0.16702316]
 [ 0.21836775  0.23316799  0.25750096 ...  0.27593922  0.28658496
   0.26486143]
 ...
 [ 0.41263763  0.12184661  0.43762542 ...  0.82412569  0.26019796
   0.59807725]
 [ 0.43429547  0.38871172  0.66850299 ...  0.58605836  0.43299575
   0.65552907]
 [ 0.26805397 -0.15460955  0.23006471 ... -0.17330118 -0.05718845
  -0.14425497]]
[[3.32539769 3.05429168 3.46964678 ... 4.40046801 4.62127351 2.8

  5%|▌         | 1/20 [00:02<00:43,  2.27s/it]

[[0.11593624 0.17204236 0.18821159 ... 0.2526619  0.22892779 0.14144574]
 [0.10520403 0.12711564 0.14695049 ... 0.18391345 0.15404262 0.14616687]
 [0.13536763 0.14727377 0.16142939 ... 0.12751558 0.14386257 0.11836724]
 ...
 [0.27473624 0.25274237 0.22886643 ... 0.21138769 0.23795606 0.23714091]
 [0.33927564 0.22396504 0.24474554 ... 0.36076321 0.29339739 0.2336602 ]
 [0.14381114 0.11776065 0.08771648 ... 0.1150288  0.14853896 0.21490608]]
[[ 0.17597889  0.1968766   0.23630167 ...  0.26813702  0.23937569
   0.24897811]
 [ 0.13991996  0.16320993  0.18410007 ...  0.16801761  0.18797507
   0.16684998]
 [ 0.21803868  0.23296019  0.25754644 ...  0.27603429  0.28664059
   0.26495408]
 ...
 [ 0.41197147  0.12208261  0.43692412 ...  0.82229681  0.26006502
   0.59698891]
 [ 0.43429547  0.38871172  0.66850299 ...  0.58605836  0.43299575
   0.65552907]
 [ 0.26574238 -0.15575707  0.22775879 ... -0.1745055  -0.05874543
  -0.14571094]]
[[3.32632957 3.05415274 3.46977818 ... 4.3969421  4.62134931 2.8

 10%|█         | 2/20 [00:04<00:40,  2.28s/it]

...

 90%|█████████ | 18/20 [00:40<00:04,  2.28s/it]

[[0.09957829 0.15638253 0.18977557 ... 0.25248811 0.22317283 0.15443284]
 [0.09820659 0.13032674 0.14295792 ... 0.18218191 0.15353341 0.14453221]
 [0.13859739 0.15163911 0.14911973 ... 0.12512845 0.1328874  0.11271958]
 ...
 [0.27662481 0.25436545 0.22581691 ... 0.20688594 0.231972   0.2311577 ]
 [0.33316281 0.24307511 0.24521619 ... 0.33195567 0.27769258 0.24088152]
 [0.14193417 0.11542222 0.09457177 ... 0.1185847  0.15196309 0.21217669]]
[[ 0.16390212  0.18921489  0.23575196 ...  0.27515386  0.24592534
   0.25312189]
 [ 0.1427688   0.16720884  0.18252087 ...  0.16782382  0.18693656
   0.1645291 ]
 [ 0.21302822  0.22997317  0.25818382 ...  0.27758358  0.28747467
   0.26637978]
 ...
 [ 0.40009928  0.12507944  0.42456291 ...  0.79121175  0.25703686
   0.57802408]
 [ 0.43429547  0.38871172  0.66850299 ...  0.58605836  0.43299575
   0.65552907]
 [ 0.22995491 -0.17201827  0.19185172 ... -0.19191177 -0.08198925
  -0.16690291]]
[[3.34364349 3.05088063 3.47233514 ... 4.33228261 4.62213818 2.6

 95%|█████████▌| 19/20 [00:42<00:02,  2.31s/it]

[[0.09867601 0.15559337 0.18981799 ... 0.25259289 0.22296106 0.1550994 ]
 [0.09787005 0.13047198 0.14279065 ... 0.18212988 0.1535295  0.14444741]
 [0.1387415  0.1519156  0.14852409 ... 0.1249818  0.1323839  0.11237289]
 ...
 [0.2767123  0.25444402 0.22564924 ... 0.20661934 0.23164854 0.23082429]
 [0.33290081 0.24402328 0.24525268 ... 0.33045392 0.27686849 0.24119585]
 [0.14178548 0.11532414 0.09495225 ... 0.11875554 0.1521133  0.2120001 ]]
[[ 0.16327661  0.18882037  0.23572824 ...  0.27551874  0.24626462
   0.25333244]
 [ 0.14291742  0.16743193  0.18243704 ...  0.16781502  0.18688223
   0.16442387]
 [ 0.21277168  0.22983144  0.2582121  ...  0.27766407  0.28751373
   0.2664505 ]
 ...
 [ 0.39937475  0.12520279  0.42381467 ...  0.78938835  0.25681898
   0.57688759]
 [ 0.43429547  0.38871172  0.66850299 ...  0.58605836  0.43299575
   0.65552907]
 [ 0.22804434 -0.1727966   0.1899257  ... -0.19276458 -0.08317553
  -0.16795214]]
[[3.34471133 3.05064233 3.47249042 ... 4.32825727 4.62214277 2.6

100%|██████████| 20/20 [00:45<00:00,  2.26s/it]


In [58]:
print(test_data_matrix.shape)

(39679, 2)


In [59]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))
rsme_v = rmse(res,train_data_matrix)
print(rsme_v)

0.877428053203302
